In [1]:
from PIL import Image
import numpy as np
from scipy import ndimage
import os
import cv2
import matplotlib.pyplot as plt
import time

import torch


RESULTS TABLE CREATION

In [61]:
results_path = '/home/kafkaon1/FVAPP/out/eval/run_230526-124959/results.npy'

In [62]:
res = np.load(results_path, allow_pickle=True).item()

In [39]:
list(res[list(res.keys())[0]].keys())

['iou', 'dice', 'dice_b1', 'dice_b3', 'dice_b5']

In [63]:
metrics = ['iou','dice', 'dice_b1','dice_b3','dice_b5']


In [64]:
table = [['metric'] + [m.replace('_','-') for m in metrics]]

for model_name, values in res.items():
    row = [model_name.replace('_','-') + '-rp']
    for metric_name in metrics:
        metric = values[metric_name]
        row.append(f"{metric.mean().item()*100:.2f}")
    table.append(row)


In [65]:
np.array(table).astype(str)

array([['metric', 'iou', 'dice', 'dice-b1', 'dice-b3', 'dice-b5'],
       ['bce-rp', '90.29', '94.83', '24.47', '54.89', '68.15'],
       ['bl-bce-a084-rp', '90.20', '94.77', '24.92', '56.17', '69.36'],
       ['bl-bdc-a089-rp', '89.89', '94.58', '24.94', '56.12', '69.22']],
      dtype='<U14')

In [66]:
np.savetxt("mydata.csv", np.array(table).astype(str), fmt="%s", delimiter=" & ", newline=" \\\\\n", header=f"\\begin{{tabular}}{{ {''.join(['c']*len(table[0]))} }}", footer="\\end{tabular}")

In [54]:
np.savetxt("mydata.csv", np.array(table).astype(str), fmt="%s", delimiter=',')

CHECK DATASET BALANCE

In [26]:
input_dir = '/home/kafkaon1/FVAPP/data/FV/'

train_data_path = os.path.join(input_dir, 'train/image_resized')
train_label_path = os.path.join(input_dir, 'train/label_resized')

val_data_path = os.path.join(input_dir, 'val/image_resized')
val_label_path = os.path.join(input_dir, 'val/label_resized')

In [129]:
def check_dataset_balance(label_names, labels_path, coeff, threshold=15000):

    max_i = int(len(label_names)*coeff)
    print(max_i)
    
    ok = []
    cnt = 0
    for i, label_name in enumerate(label_names[:max_i]):
        label_path = os.path.join(labels_path, label_name)
        label = np.array(Image.open(label_path))
 
        cnt += (label == 1).sum() < threshold
        if (label == 1).sum() >= threshold:
            ok.append(label_name)
            #display(Image.fromarray(np.uint8(label*100)))


    print(f"{cnt}:{cnt/i*100:.2f}%")
    return ok

In [146]:
with open('/home/kafkaon1/FVAPP/data/FV/vala.txt') as f:
    names = f.read()
    f.close()
    
names = names.split('\n')[:-1]

In [147]:
len(names)

23267

In [148]:
ok= check_dataset_balance(names ,val_label_path, 1)

23267
0:0.00%


In [144]:
k = '\n'.join(ok)

In [145]:
with open('/home/kafkaon1/FVAPP/data/FV/vala.txt', "w") as fo:
    fo.writelines(k)
    fo.close()

In [108]:
with open('/home/kafkaon1/FVAPP/data/FV/val.txt') as f:
    train_names = f.readlines()
    f.close()

np.random.shuffle(train_names)

with open('/home/kafkaon1/FVAPP/data/FV/vala.txt', "a") as fo:
    fo.writelines(train_names)
    fo.close()
